In [5]:
import networkx as nx
import pyalex
from pyalex import Works, Institutions, Topics, Funders
from collections import Counter
import matplotlib.pyplot as plt
from collections import defaultdict
from itertools import chain
import numpy as np
import scipy
pyalex.config.email = "aymann.mhammedi@gmail.com"

In [6]:
# Fetch a list of topics
topics = Topics().get()

# Print the list of topics with their IDs and display names
#for topic in topics:
#   print(f"ID: {topic['id']}, Display Name: {topic['display_name']}, Description: {topic['description']}")

In [7]:
funders = Funders().get()
# List fields of funders object
print(funders[0].keys())

dict_keys(['id', 'display_name', 'alternate_titles', 'country_code', 'description', 'homepage_url', 'image_url', 'image_thumbnail_url', 'grants_count', 'works_count', 'cited_by_count', 'summary_stats', 'ids', 'counts_by_year', 'roles', 'updated_date', 'created_date'])


In [ ]:
# Define the filter criteria for works published in the USA in 2015
filter_criteria = {
    "from_publication_date": "2016-01-01",  # Start date
    "to_publication_date": "2020-12-31",    # End date
    "institutions": {"country_code": "us"},  # Filter for U.S.-based institutions
    "primary_location": {"source": {"type": "journal"}},  # Filter for journal articles
    "primary_topic": {"subfield": {"id": 2740}},  # Filter for Pulmonary and Respiratory Medicine
}


# Fetch the works with pagination
query = Works().filter(**filter_criteria)
all_works_16_20 = list(chain.from_iterable(query.paginate(per_page=200, n_max=None)))  # Adjust per_page as needed, n_max=None for all papers (heavy)

# Count the number of works
num_works_16_20 = len(all_works_16_20)

# Print the number of works
print(f"Number of works published in the USA in between 2016-2020: {num_works_16_20}")

# Optionally, print the first few works to verify
for work in all_works_16_20[:5]:  # Print the first 5 works as an example
    print(f"ID: {work['id']}, Title: {work['title']}, Publication Year: {work['publication_year']}")

In [ ]:
# Create all_works_2018 with only works from 2018
all_works_2018 = [work for work in all_works_16_20 if work['publication_year'] == 2018]
num_works = len(all_works_2018)

In [ ]:
# Count the number of unique work IDs
unique_work_ids = len(set(work['id'] for work in all_works_2018))

# Print the result
print(f"Number of unique work IDs: {unique_work_ids}")

## Collaboration coauthorship

In [ ]:
# Construct a collaboration network with country information
G_collaboration = nx.Graph()

# Add nodes and edges for authors
for work in all_works_2018:
    authors = [
        (authorship['author']['id'], authorship['countries'])  # Assuming the first country is the primary one
        for authorship in work['authorships']
    ]
    for i, (author1, country1) in enumerate(authors):
        # Add author1 node with country attribute
        G_collaboration.add_node(author1, country=country1)
        for author2, country2 in authors[i + 1:]:
            # Add author2 node with country attribute
            G_collaboration.add_node(author2, country=country2)
            # Add edge between author1 and author2
            if G_collaboration.has_edge(author1, author2):
                G_collaboration[author1][author2]['weight'] += 1
            else:
                G_collaboration.add_edge(author1, author2, weight=1)

COAUTHORSHIP

In [ ]:
# Number of nodes and edges in the collaboration network
num_nodes = G_collaboration.number_of_nodes()
num_edges = G_collaboration.number_of_edges()
print(f"Number of authors in the collaboration network: {num_nodes}")
print(f"Number of edges in the collaboration network: {num_edges}")
# Mean and median edges per node
mean_edges_per_node = np.mean([G_collaboration.degree(n) for n in G_collaboration.nodes()])
median_edges_per_node = np.median([G_collaboration.degree(n) for n in G_collaboration.nodes()])
print(f"Mean number of coauthors per author: {mean_edges_per_node}")
print(f"Median number of coauthors per author: {median_edges_per_node}")

In [ ]:
# Degree centrality
#degree_centrality = nx.degree_centrality(G_collaboration)
#top_collaborators = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:10]
#print("Top collaborative authors by degree centrality:", top_collaborators)

# Betweenness centrality
#betweenness_centrality = nx.betweenness_centrality(G_collaboration)
#top_intermediaries = sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)[:10]
#print("Top intermediaries in the network by betweenness centrality:", top_intermediaries)

# Connected components
#connected_components = list(nx.connected_components(G_collaboration))
#print(f"Number of connected components: {len(connected_components)}")

In [ ]:
# Count the number of coauthors for each work
coauthor_counts = Counter(len(work['authorships']) for work in all_works_2018)

# Prepare data for plotting
coauthor_numbers = list(coauthor_counts.keys())
paper_counts = list(coauthor_counts.values())

# Plot the data
plt.figure(figsize=(10, 6))
plt.bar(coauthor_numbers, paper_counts, color='skyblue')
plt.xlabel('Number of Coauthors')
plt.ylabel('Number of Papers')
plt.title('Number of Papers for Each Number of Coauthors')
plt.xticks(coauthor_numbers)
plt.show()

In [ ]:
#Mean and median number of coauthors per paper
mean_coauthors = np.mean([len(work['authorships']) for work in all_works_2018])
median_coauthors = np.median([len(work['authorships']) for work in all_works_2018])
print(f"Mean number of coauthors per paper: {mean_coauthors:.2f}")
print(f"Median number of coauthors per paper: {median_coauthors}")

### Exposure to China

J'arrive pas à ajouter le pays dans le graph des auteurs

## Graph of Works

In [ ]:
# Contruct a graph of funders and papers (nodes are funders and papers, edges are funding)
G_works = nx.Graph()
# Add nodes for each work and its funders
for work in all_works_2018:
    work_id = work['id']
    G_works.add_node(work_id, type='work')
    for grant in work['grants']:
        funder_name = grant['funder_display_name']
        G_works.add_node(funder_name, type='funder')
        G_works.add_edge(work_id, funder_name)
        # Add country to the funder type nodes
        if 'country_code' in grant:
            G_works.nodes[funder_name]['country'] = grant['country_code']
        else:
            # If country_code is not available, set a default value or skip
            G_works.nodes[funder_name]['country'] = 'Unknown'
# Add attributes to work nodes (citation count, has_chinese_coauthor, has_grant, etc.)
for work in all_works_2018:
    work_id = work['id']
    G_works.nodes[work_id]['cited_by_count'] = work['cited_by_count']
    G_works.nodes[work_id]['has_grant'] = len(work['grants']) > 0
# Add nodes for authors and edges between authors and papers
    for authorship in work['authorships']:
        author_id = authorship['author']['id']
        G_works.add_node(author_id, type='author')
        G_works.add_edge(work_id, author_id)
# Add attribute to work nodes (has_chinese_coauthor, has_not_american_coauthor, etc.)
    G_works.nodes[work_id]['has_not_american_coauthor'] = any('US' not in authorship['countries'] for authorship in work['authorships'])
    G_works.nodes[work_id]['has_chinese_coauthor'] = any('CN' in authorship['countries'] for authorship in work['authorships'])


### International collaboration

In [ ]:
# Share of work nodes with only american coauthors, at least 1 foreign coauthor, at least 1 chinese coauthor
share_american_coauthors = sum(1 for work in all_works_2018 if G_works.nodes[work['id']]['has_not_american_coauthor']) / len(all_works_2018)
share_foreign_coauthors = sum(1 for work in all_works_2018 if G_works.nodes[work['id']]['has_not_american_coauthor'] == 0 ) / len(all_works_2018)
share_chinese_coauthors = sum(1 for work in all_works_2018 if G_works.nodes[work['id']]['has_chinese_coauthor']) / len(all_works_2018)
print(f"Share of works with only American coauthors: {share_american_coauthors:.2%}")
print(f"Share of works with at least one foreign coauthor: {share_foreign_coauthors:.2%}")
print(f"Share of works with at least one Chinese coauthor: {share_chinese_coauthors:.2%}")

### Quality of work

In [ ]:
# Mean and median citation counts for works with or without international coauthors
mean_citations_with_international = np.mean([G_works.nodes[work['id']]['cited_by_count'] for work in all_works_2018 if G_works.nodes[work['id']]['has_not_american_coauthor'] == 0])
mean_citations_without_international = np.mean([G_works.nodes[work['id']]['cited_by_count'] for work in all_works_2018 if G_works.nodes[work['id']]['has_not_american_coauthor']])
median_citations_with_international = np.median([G_works.nodes[work['id']]['cited_by_count'] for work in all_works_2018 if G_works.nodes[work['id']]['has_not_american_coauthor'] == 0])
median_citations_without_international = np.median([G_works.nodes[work['id']]['cited_by_count'] for work in all_works_2018 if G_works.nodes[work['id']]['has_not_american_coauthor']])
print(f"Mean citation count for works with international coauthors: {mean_citations_with_international:.2f}")
print(f"Mean citation count for works without international coauthors: {mean_citations_without_international:.2f}")
print(f"Median citation count for works with international coauthors: {median_citations_with_international:.2f}")
print(f"Median citation count for works without international coauthors: {median_citations_without_international:.2f}")

In [ ]:
# Mean and median citation counts for works with and without chinese coauthors
mean_citations_with_chinese = np.mean([G_works.nodes[work['id']]['cited_by_count'] for work in all_works_2018 if G_works.nodes[work['id']]['has_chinese_coauthor']])
mean_citations_without_chinese = np.mean([G_works.nodes[work['id']]['cited_by_count'] for work in all_works_2018 if G_works.nodes[work['id']]['has_chinese_coauthor'] == 0])
median_citations_with_chinese = np.median([G_works.nodes[work['id']]['cited_by_count'] for work in all_works_2018 if G_works.nodes[work['id']]['has_chinese_coauthor']])
median_citations_without_chinese = np.median([G_works.nodes[work['id']]['cited_by_count'] for work in all_works_2018 if G_works.nodes[work['id']]['has_chinese_coauthor'] == 0])
print(f"Mean citation count for works with Chinese coauthors: {mean_citations_with_chinese:.2f}")
print(f"Mean citation count for works without Chinese coauthors: {mean_citations_without_chinese:.2f}")
print(f"Median citation count for works with Chinese coauthors: {median_citations_with_chinese:.2f}")
print(f"Median citation count for works without Chinese coauthors: {median_citations_without_chinese:.2f}")

### Fundings

In [ ]:
# Among US-only papers, at least 1 foreign coauthor, at least 1 Chinese coauthor: what share has a grant?
share_grant_with_international = sum(1 for work in all_works_2018 if G_works.nodes[work['id']]['has_not_american_coauthor'] == 0 and G_works.nodes[work['id']]['has_grant']) / sum(1 for work in all_works_2018 if G_works.nodes[work['id']]['has_not_american_coauthor'] == 0)
share_grant_without_international = sum(1 for work in all_works_2018 if G_works.nodes[work['id']]['has_not_american_coauthor'] == 1 and G_works.nodes[work['id']]['has_grant']) / sum(1 for work in all_works_2018 if G_works.nodes[work['id']]['has_not_american_coauthor'] == 1)
share_grant_with_chinese = sum(1 for work in all_works_2018 if G_works.nodes[work['id']]['has_chinese_coauthor'] and G_works.nodes[work['id']]['has_grant']) / sum(1 for work in all_works_2018 if G_works.nodes[work['id']]['has_chinese_coauthor'])

print(f"Share of works with international coauthors that have a grant: {share_grant_with_international:.2%}")
print(f"Share of works without international coauthors that have a grant: {share_grant_without_international:.2%}")
print(f"Share of works with Chinese coauthors that have a grant: {share_grant_with_chinese:.2%}")

In [ ]:
# For works with at least 1 chinese coauthor, who are the funders?
USCN_funder_counts = Counter()
for work in all_works_2018:
    if G_works.nodes[work['id']]['has_chinese_coauthor']:
        for grant in work['grants']:
            funder_name = grant['funder_display_name']
            USCN_funder_counts[funder_name] += 1
# Print the top 10 funders for works with at least one Chinese coauthor
print("Funders for works with at least one Chinese coauthor:")
for funder, count in USCN_funder_counts.most_common(10):  # Print top 10 funders
    print(f"{funder}: {count} works")

In [ ]:
# For works with at least 1 chinese coauthor, from which countries are the funders?
USCN_funder_country_counts = Counter()
for work in all_works_2018:
    if G_works.nodes[work['id']]['has_chinese_coauthor']:
        for grant in work['grants']:
            funder_country = G_works.nodes[grant['funder_display_name']]['country']
            USCN_funder_country_counts[funder_country] += 1
# Print the top 10 funder countries for works with at least one Chinese coauthor
print("Funder countries for works with at least one Chinese coauthor:")
for country, count in USCN_funder_country_counts.most_common(10):  # Print top 10 funder countries
    print(f"{country}: {count} works")

In [ ]:
# Density of the subgraph
density = nx.density(us_china_collaboration)
print(f"Density of U.S.-China collaboration subgraph: {density}")

# Bridges in the subgraph
bridges = list(nx.bridges(us_china_collaboration))
print(f"Number of bridges in U.S.-China collaboration subgraph: {len(bridges)}")
# edges whose removal disconnects the graph to find critical collaborations.

In [ ]:
# Get total number of unique institutions
unique_institutions = set()
for work in all_works_2018:
    for author in work['authorships']:
        for institution in author['institutions']:
            unique_institutions.add(institution['id'])
print(f"Total number of unique institutions: {len(unique_institutions)}")

In [ ]:
# Plot number of unique institutions per paper
institution_counts = Counter(len(work['authorships'][0]['institutions']) for work in all_works_2018)
institution_numbers = list(institution_counts.keys())
institution_paper_counts = list(institution_counts.values())
plt.figure(figsize=(10, 6))
plt.bar(institution_numbers, institution_paper_counts, color='salmon')
plt.xlabel('Number of Institutions')

In [ ]:
# For each work generate dummy variable if at least one coauthor is from china, add it to all_works_2018
for work in all_works_2018:
    work['has_chinese_coauthor'] = any('CN' in author['countries'] for author in work['authorships'])
# Count the number of works with at least one Chinese coauthor
num_works_with_chinese_coauthor = sum(work['has_chinese_coauthor'] for work in all_works_2018)
print(f"Number of works with at least one Chinese coauthor: {num_works_with_chinese_coauthor}")
print(f"Percentage of works with at least one Chinese coauthor: {num_works_with_chinese_coauthor / num_works * 100:.2f}%")

In [ ]:
# Count the number of Chinese coauthors
chinese_coauthors = sum(
    1
    for work in all_works_2018
    for authorship in work['authorships']
    for inst in authorship.get('institutions', [])
    if inst.get('country_code') == 'CN'
)

# Print the result
print(f"Number of Chinese coauthors: {chinese_coauthors}")

In [ ]:
#For each work generate dummy variable if all authors are from USA, add it to all_works_2018
for work in all_works_2018:
    work['all_authors_from_usa'] = all('US' in author['countries'] for author in work['authorships'])
# Count the number of works with all authors from the USA
num_works_all_authors_from_usa = sum(work['all_authors_from_usa'] for work in all_works_2018)
print(f"Number of works with all authors from the USA: {num_works_all_authors_from_usa}")

## QUALITY OF RESEARCH

In [ ]:
# Get min max mean and deciles of cited_by_count
cited_by_counts = [work['cited_by_count'] for work in all_works_2018]
min_cited_by_count = min(cited_by_counts)
max_cited_by_count = max(cited_by_counts)
mean_cited_by_count = np.mean(cited_by_counts)
deciles = np.percentile(cited_by_counts, np.arange(0, 101, 10))
print(f"Min cited_by_count: {min_cited_by_count}")
print(f"Max cited_by_count: {max_cited_by_count}")
print(f"Mean cited_by_count: {mean_cited_by_count:.2f}")
print(f"Deciles of cited_by_count: {deciles}")

In [ ]:
# Distribution of citations per paper
citations_counts = Counter(work['cited_by_count'] for work in all_works_2018)
# Prepare data for plotting
citation_numbers = list(citations_counts.keys())
citation_paper_counts = list(citations_counts.values())
# Plot the data
plt.figure(figsize=(10, 6))
plt.xlim(0, 100)
plt.ylim(0, 2000)
plt.bar(citation_numbers, citation_paper_counts, color='lightgreen')

In [ ]:
#Mean citations for paperss with and without Chinese coauthors
mean_citations_with_chinese = np.mean([work['cited_by_count'] for work in all_works_2018 if work['has_chinese_coauthor']])
mean_citations_without_chinese = np.mean([work['cited_by_count'] for work in all_works_2018 if not work['has_chinese_coauthor']])
print(f"Mean citations for papers with Chinese coauthors: {mean_citations_with_chinese:.2f}")
print(f"Mean citations for papers without Chinese coauthors: {mean_citations_without_chinese:.2f}")

### EXPOSURE TO POLICY

In [ ]:
# Count the number of papers per author
author_paper_counts = Counter(
    authorship['author']['id']
    for work in all_works_2018
    for authorship in work['authorships']
)

# Prepare data for plotting
authors = list(author_paper_counts.keys())
paper_counts = list(author_paper_counts.values())

# Plot the data
plt.figure(figsize=(10, 6))
plt.hist(paper_counts, bins=range(1, max(paper_counts) + 2), color='skyblue', edgecolor='black')
plt.xlabel('Number of Papers')
plt.ylabel('Number of Authors')
plt.title('Distribution of Number of Papers per Author')
plt.show()

In [ ]:
# For each US author compute share of papers with Chinese coauthors
for work in all_works_2018:
    for authorship in work['authorships']:
        if 'US' in authorship['countries']:
            authorship['has_chinese_coauthor'] = any('CN' in author['countries'] for author in work['authorships'])
# Count the number of papers for each US author
us_author_paper_counts = defaultdict(int)
for work in all_works_2018:
    for authorship in work['authorships']:
        if 'US' in authorship['countries']:
            us_author_paper_counts[authorship['author']['id']] += 1
# Compute the share of papers with Chinese coauthors for each US author
us_author_chinese_coauthor_counts = defaultdict(int)
for work in all_works_2018:
    for authorship in work['authorships']:
        if 'US' in authorship['countries'] and authorship.get('has_chinese_coauthor'):
            us_author_chinese_coauthor_counts[authorship['author']['id']] += 1
# Compute the share of papers with Chinese coauthors for each US author
us_author_shares = {
    author_id: us_author_chinese_coauthor_counts[author_id] / count
    for author_id, count in us_author_paper_counts.items()
}
# Prepare data for plotting
us_author_ids = list(us_author_shares.keys())
us_author_share_values = list(us_author_shares.values())
# Plot the data
plt.figure(figsize=(10, 6))
plt.hist(us_author_share_values, bins=20, color='salmon', edgecolor='black')
plt.xlabel('Share of Papers with Chinese Coauthors')
plt.ylabel('Number of US Authors')
plt.title('Distribution of Share of Papers with Chinese Coauthors for US Authors')

In [ ]:
#List US authors with share of papers with Chinese coauthors > 0.5
high_share_authors = [author_id for author_id, share in us_author_shares.items() if share > 0.5]

## CHINA INITIATIVE

In [ ]:
# Define the filter criteria for works published in the USA in 2020
filter_criteria = {
    "publication_year": 2019,  # Filter for works published in 2020
    "institutions": {"country_code": "us"},
    "primary_location": {"source": {"type": "journal"}},  # Filter for journal articles
    #"primary_topic": {"domain": {"id": 4}},  # Filter for Health science
    "primary_topic": {"subfield": {"id": 2740}},  # Filter for Pulmonary and Respiratory Medicine
}

# Fetch the works with pagination
query = Works().filter(**filter_criteria)
all_works_2019 = list(chain.from_iterable(query.paginate(per_page=200, n_max=None)))  # Adjust per_page as needed, n_max=None for all papers (heavy)

# Count the number of works
num_works = len(all_works_2019)

# Print the number of works
print(f"Number of works published in the USA in 2020: {num_works}")

# Optionally, print the first few works to verify
for work in all_works_2019[:5]:  # Print the first 5 works as an example
    print(f"ID: {work['id']}, Title: {work['title']}, Publication Year: {work['publication_year']}")

In [ ]:
# For each work generate dummy variable if at least one coauthor is from china, add it to all_works_2020
for work in all_works_2019:
    work['has_chinese_coauthor'] = any('CN' in author['countries'] for author in work['authorships'])
# Count the number of works with at least one Chinese coauthor
num_works_with_chinese_coauthor = sum(work['has_chinese_coauthor'] for work in all_works_2019)
print(f"Number of works with at least one Chinese coauthor: {num_works_with_chinese_coauthor}")
print(f"Percentage of works with at least one Chinese coauthor: {num_works_with_chinese_coauthor / num_works * 100:.2f}%")

In [ ]:
all_works_2019[0]['grants']

In [ ]:
len(all_works_2019[0]['grants'])

In [ ]:
# Add dummy variable for each work if it has a grant
for work in all_works_2019:
    work['has_grant'] = len(work['grants']) > 0
# Count the number of works with at least one grant
num_works_with_grant = sum(work['has_grant'] for work in all_works_2019)
print(f"Number of works with at least one grant: {num_works_with_grant}")

In [ ]:
# Plot size of grants per paper
grant_counts = Counter(len(work['grants']) for work in all_works_2019)
# Prepare data for plotting
grant_numbers = list(grant_counts.keys())
grant_paper_counts = list(grant_counts.values())
# Plot the data
plt.figure(figsize=(10, 6))
plt.bar(grant_numbers, grant_paper_counts, color='lightgreen')
plt.xlabel('Number of Grants')
plt.ylabel('Number of Papers')
plt.title('Number of Papers for Each Number of Grants')

In [ ]:
# Comprend pas pourquoi ca marche pas (voir cellule suivante -> il y a grants)
all_works_2019['id' == 'https://openalex.org/W2990041408']['has_grant']

In [ ]:
# Print works with grants
#for work in all_works_2019:
#   if len(work['grants']) > 0:
#        print(f"ID: {work['id']}, Has Grant: {work['has_grant']}, Grant: {work['grants']}")

In [ ]:
# Print funder names and their counts
funder_counts = Counter()
for work in all_works_2019:
    for grant in work['grants']:
        funder_counts[grant['funder_display_name']] += 1
# Print the funder names and their counts
for funder, count in funder_counts.most_common(10):  # Print top 10 funders
    print(f"Funder: {funder}, Count: {count}")

In [ ]:
# Number of funders in funder_counts
num_funders = len(funder_counts)
print(f"Number of funders: {num_funders}")

In [ ]:
# Top funders of works with Chinese coauthors vs without Chinese coauthors
funder_counts_with_chinese = Counter()
funder_counts_without_chinese = Counter()
for work in all_works_2019:
    for grant in work['grants']:
        if work['has_chinese_coauthor']:
            funder_counts_with_chinese[grant['funder_display_name']] += 1
        else:
            funder_counts_without_chinese[grant['funder_display_name']] += 1
# Print the top funders for works with Chinese coauthors
print("Top funders for works with Chinese coauthors:")
for funder, count in funder_counts_with_chinese.most_common(10):  # Print top 10 funders
    print(f"Funder: {funder}, Count: {count}")
# Print the top funders for works without Chinese coauthors
print("Top funders for works without Chinese coauthors:")
for funder, count in funder_counts_without_chinese.most_common(10):  # Print top 10 funders
    print(f"Funder: {funder}, Count: {count}")

In [ ]:
# Top funders of works with Chinese coauthors vs without Chinese coauthors
funder_counts_with_chinese = Counter()
funder_counts_without_chinese = Counter()
for work in all_works_2018:

    for grant in work['grants']:
        if work['has_chinese_coauthor']:
            funder_counts_with_chinese[grant['funder_display_name']] += 1
        else:
            funder_counts_without_chinese[grant['funder_display_name']] += 1
# Print the top funders for works with Chinese coauthors
print("Top funders for works with Chinese coauthors:")
for funder, count in funder_counts_with_chinese.most_common(10):  # Print top 10 funders
    print(f"Funder: {funder}, Count: {count}")
# Print the top funders for works without Chinese coauthors
print("Top funders for works without Chinese coauthors:")
for funder, count in funder_counts_without_chinese.most_common(10):  # Print top 10 funders
    print(f"Funder: {funder}, Count: {count}")

Introducing more info on fundings

## Construct networks

In [ ]:
# Count number of funder nodes in the graph
num_funder_nodes = sum(1 for node, data in G_fundings.nodes(data=True) if data['type'] == 'funder')
print(f"Number of funder nodes in the graph: {num_funder_nodes}")

In [ ]:
# Count number of author nodes in the graph
num_author_nodes = sum(1 for node, data in G_fundings.nodes(data=True) if data['type'] == 'author')
print(f"Number of author nodes in the graph: {num_author_nodes}")

In [ ]:
# Count the number of authors for each work and store in all_works_2019
for work in all_works_2019:
    work['num_authors'] = len(work['authorships'])
#get total number of authors
total_coauthors = sum(work['num_authors'] for work in all_works_2019)
#get total of unique authors
unique_authors = set()
for work in all_works_2019:
    for author in work['authorships']:
        unique_authors.add(author['author']['id'])

print(f"Total number of authors: {total_coauthors}")
print(f"Total number of unique authors: {len(unique_authors)}")

SURPLUS

In [ ]:
# Extract all unique country codes of co-authors
country_codes = set(
    inst.get('country_code') 
    for work in all_works_2018 
    for authorship in work['authorships'] 
    for inst in authorship.get('institutions', []) 
    if 'country_code' in inst
)

# Print the unique country codes
print(f"Different country codes of co-authors: {country_codes}")

In [ ]:
# Count institutions in the US and outside the US
us_institutions = set(
    inst['id']
    for work in all_works_2018
    for authorship in work['authorships']
    for inst in authorship.get('institutions', [])
    if inst.get('country_code') == 'US'
)

non_us_institutions = set(
    inst['id']
    for work in all_works_2018
    for authorship in work['authorships']
    for inst in authorship.get('institutions', [])
    if inst.get('country_code') != 'US'
)

cn_institutions = set(
    inst['id']
    for work in all_works_2018
    for authorship in work['authorships']
    for inst in authorship.get('institutions', [])
    if inst.get('country_code') == 'CN'
)

# Print the results
print(f"Number of institutions in the US: {len(us_institutions)}")
print(f"Number of institutions outside the US: {len(non_us_institutions)}")
print(f"Number of institutions in China: {len(cn_institutions)}")

## Assessing first order effects 

In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict

# ========================
# Step 1: Select Top 10% of China-collaborating US researchers (before Nov 2018)
# ========================

# Build US authors + share of papers with Chinese coauthors
us_author_total_papers_pre = defaultdict(int)
us_author_chinese_papers_pre = defaultdict(int)

for work in all_works_16_20:  # assuming you already have all works 2016-2020
    if work['publication_year'] < 2019:  # Before Nov 2018 => anything up to 2018
        for authorship in work['authorships']:
            if 'US' in authorship['countries']:
                us_author_total_papers_pre[authorship['author']['id']] += 1
                if any('CN' in coauth['countries'] for coauth in work['authorships']):
                    us_author_chinese_papers_pre[authorship['author']['id']] += 1

# Compute share of China collaboration
us_author_china_share = {
    author_id: us_author_chinese_papers_pre[author_id] / us_author_total_papers_pre[author_id]
    for author_id in us_author_total_papers_pre
    if us_author_total_papers_pre[author_id] >= 2  # filter for active researchers
}

# Find top 10% most China-collaborative authors
threshold = np.percentile(list(us_author_china_share.values()), 90)
top_china_authors = {author_id for author_id, share in us_author_china_share.items() if share >= threshold}

print(f"Selected {len(top_china_authors)} top China-collaborating US authors.")

# ========================
# Step 2: Track outcomes before vs after Nov 2018
# ========================

# Initialize metrics
author_metrics = defaultdict(lambda: {
    'pre_pubs': 0, 'post_pubs': 0,
    'pre_china_pubs': 0, 'post_china_pubs': 0,
    'pre_other_foreign_pubs': 0, 'post_other_foreign_pubs': 0,
    'pre_citations': [], 'post_citations': []
})

for work in all_works_16_20:
    year = work['publication_year']
    if year < 2016 or year > 2020:
        continue  # skip

    is_pre = (year <= 2018)  # Pre-shock = 2016–2018
    is_post = (year >= 2019)  # Post-shock = 2019–2020

    has_chinese_coauthor = any('CN' in auth['countries'] for auth in work['authorships'])
    has_other_foreign_coauthor = any(
        c not in ['US', 'CN'] for auth in work['authorships'] for c in auth['countries']
    )

    for authorship in work['authorships']:
        author_id = authorship['author']['id']
        if author_id in top_china_authors:
            if is_pre:
                author_metrics[author_id]['pre_pubs'] += 1
                if has_chinese_coauthor:
                    author_metrics[author_id]['pre_china_pubs'] += 1
                if has_other_foreign_coauthor:
                    author_metrics[author_id]['pre_other_foreign_pubs'] += 1
                author_metrics[author_id]['pre_citations'].append(work['cited_by_count'])
            elif is_post:
                author_metrics[author_id]['post_pubs'] += 1
                if has_chinese_coauthor:
                    author_metrics[author_id]['post_china_pubs'] += 1
                if has_other_foreign_coauthor:
                    author_metrics[author_id]['post_other_foreign_pubs'] += 1
                author_metrics[author_id]['post_citations'].append(work['cited_by_count'])

# ========================
# Step 3: Prepare clean table
# ========================

# Build a dataframe
rows = []
for author_id, metrics in author_metrics.items():
    rows.append({
        'author_id': author_id,
        'pre_pubs': metrics['pre_pubs'],
        'post_pubs': metrics['post_pubs'],
        'pre_china_share': metrics['pre_china_pubs'] / metrics['pre_pubs'] if metrics['pre_pubs'] > 0 else np.nan,
        'post_china_share': metrics['post_china_pubs'] / metrics['post_pubs'] if metrics['post_pubs'] > 0 else np.nan,
        'pre_foreign_share': metrics['pre_other_foreign_pubs'] / metrics['pre_pubs'] if metrics['pre_pubs'] > 0 else np.nan,
        'post_foreign_share': metrics['post_other_foreign_pubs'] / metrics['post_pubs'] if metrics['post_pubs'] > 0 else np.nan,
        'pre_mean_citations': np.mean(metrics['pre_citations']) if metrics['pre_citations'] else np.nan,
        'post_mean_citations': np.mean(metrics['post_citations']) if metrics['post_citations'] else np.nan,
    })

df_metrics = pd.DataFrame(rows)

# ========================
# Step 4: Analysis
# ========================

# Change in collaborations
df_metrics['delta_china_share'] = df_metrics['post_china_share'] - df_metrics['pre_china_share']
df_metrics['delta_foreign_share'] = df_metrics['post_foreign_share'] - df_metrics['pre_foreign_share']

# Change in publications
df_metrics['delta_pubs'] = df_metrics['post_pubs'] - df_metrics['pre_pubs']

# Change in citations
df_metrics['delta_mean_citations'] = df_metrics['post_mean_citations'] - df_metrics['pre_mean_citations']

# Summary
print(df_metrics.describe())

# You can plot histograms of the deltas:
import matplotlib.pyplot as plt

plt.hist(df_metrics['delta_china_share'].dropna(), bins=20, alpha=0.7)
plt.title('Change in China Collaboration Share (Post-Pre)')
plt.xlabel('Delta Share')
plt.ylabel('Number of Authors')
plt.show()

plt.hist(df_metrics['delta_foreign_share'].dropna(), bins=20, alpha=0.7)
plt.title('Change in Other Foreign Collaboration Share (Post-Pre)')
plt.xlabel('Delta Share')
plt.ylabel('Number of Authors')
plt.show()


## Trying to plot the graph with datashader

In [ ]:
# === INSTALL IF NEEDED ===
# !pip install -U datashader holoviews bokeh

# === IMPORTS ===
import holoviews as hv
import datashader as ds
import datashader.transfer_functions as tf
import networkx as nx
import pandas as pd
import numpy as np

hv.extension('bokeh')

# === STEP 1: BUILD SPRING LAYOUT ===
print("Computing layout for G_collaboration... (this can take a few minutes)")
try:
    pos = nx.spring_layout(G_collaboration, seed=42, k=1/np.sqrt(G_collaboration.number_of_nodes()))
except MemoryError:
    print("MemoryError: using spectral layout instead...")
    pos = nx.spectral_layout(G_collaboration)

# Create DataFrames for nodes and edges
print("Building DataFrames...")
nodes_df = pd.DataFrame({
    'x': [pos[node][0] for node in G_collaboration.nodes()],
    'y': [pos[node][1] for node in G_collaboration.nodes()],
})

edges_x = []
edges_y = []
for u, v in G_collaboration.edges():
    x0, y0 = pos[u]
    x1, y1 = pos[v]
    edges_x.append([x0, x1])
    edges_y.append([y0, y1])

edges_df = pd.DataFrame({
    'x0': [p[0] for p in edges_x],
    'y0': [p[0] for p in edges_y],
    'x1': [p[1] for p in edges_x],
    'y1': [p[1] for p in edges_y],
})

# === STEP 2: PLOT USING DATASHADER ===
print("Rendering with Datashader...")

canvas = ds.Canvas(plot_width=1200, plot_height=1200)

# Plot edges
agg_edges = canvas.line(edges_df, 'x0', 'y0', 'x1', 'y1', agg=ds.count())

# Plot nodes separately (optional, because usually edges dominate)
agg_nodes = canvas.points(nodes_df, 'x', 'y', agg=ds.count())

# Combine edges and nodes
img_edges = tf.shade(agg_edges, cmap=["lightblue", "darkblue"], how='log')
img_nodes = tf.shade(agg_nodes, cmap=["orange", "red"], how='log')

final_img = tf.stack(img_edges, img_nodes)

# Display
hv.output(size=200)
hv.Image(final_img)